In [1]:
#from google.colab import drive
#drive.mount('/content/drive')

In [2]:
#import sys
#sys.path.append('/content/drive/MyDrive/Transformer-BasedGPTModel')

In [3]:
!pip install dill
import dill as pickle
import torch
from GPTLanguageModel import GPTLanguageModel
import os

Defaulting to user installation because normal site-packages is not writeable


In [4]:
batch_size = 64
block_size = 256 # maximum number of tokens the model can use as input at once when making predictions.
max_iters = 5000
eval_interval = 20
learning_rate = 3e-6
device = 'cuda' if torch.cuda.is_available() else 'cpu'
eval_iters = 200
torch.manual_seed(1337)

In [5]:
with open('C:\\Users\\NiKordzakhia\\Desktop\\Transformer-BasedGPTModel\\vefxistyaosani.txt', 'r', encoding='utf-8') as f:
    text = f.read()

In [6]:
with open('C:\\Users\\NiKordzakhia\\Desktop\\Transformer-BasedGPTModel\\tokenizer', 'rb') as f:
    tokenizer = pickle.load(f)

In [7]:
#data = torch.tensor(tokenizer.encode(text), dtype=torch.long)
#torch.save(data, 'C:\\Users\\NiKordzakhia\\Desktop\\Transformer-BasedGPTModel\\data.pt')
data = torch.load('C:\\Users\\NiKordzakhia\\Desktop\\Transformer-BasedGPTModel\\data.pt')
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [8]:
n = int(0.9*len(data))
train_data = data[:n]
val_data = data[n:]

In [9]:
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x, y

In [10]:
x, y = get_batch('train')
print('inputs:')
print(x.shape)
print('targets:')
print(y.shape)

inputs:
torch.Size([64, 256])
targets:
torch.Size([64, 256])


In [11]:
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [15]:
model = GPTLanguageModel()
print(model)
m = model.to(device)
print(sum(p.numel() for p in m.parameters())/1e6, 'M parameters')

GPTLanguageModel(
  (token_embedding_table): Embedding(7000, 256)
  (position_embedding_table): Embedding(256, 256)
  (blocks): Sequential(
    (0): Block(
      (sa): MultiHeadAttention(
        (heads): ModuleList(
          (0-15): 16 x Head(
            (key): Linear(in_features=256, out_features=16, bias=False)
            (query): Linear(in_features=256, out_features=16, bias=False)
            (value): Linear(in_features=256, out_features=16, bias=False)
            (dropout): Dropout(p=0.2, inplace=False)
          )
        )
        (proj): Linear(in_features=256, out_features=256, bias=True)
        (dropout): Dropout(p=0.2, inplace=False)
      )
      (ffwd): FeedFoward(
        (net): Sequential(
          (0): Linear(in_features=256, out_features=1024, bias=True)
          (1): ReLU()
          (2): Linear(in_features=1024, out_features=256, bias=True)
          (3): Dropout(p=0.2, inplace=False)
        )
      )
      (ln1): LayerNorm()
      (ln2): LayerNorm()
    )
 

In [13]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

In [ ]:
for iter in range(max_iters):
    if iter % eval_interval == 0 or iter == max_iters - 1:
        losses = estimate_loss()
        print(f"step {iter}: train loss {losses['train']:.4f}, val loss {losses['val']:.4f}")

    xb, yb = get_batch('train')

    logits, loss = model(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()

torch.Size([16384, 7000]) torch.Size([16384])
torch.Size([16384, 7000]) torch.Size([16384])


In [18]:
with open('C:\\Users\\NiKordzakhia\\Desktop\\Transformer-BasedGPTModel\\GPTModel', 'wb') as f:
    pickle.dump(model, f)

with open('C:\\Users\\NiKordzakhia\\Desktop\\Transformer-BasedGPTModel\\GPTModel', 'rb') as f:
    gpt_model = pickle.load(f)

In [21]:
context = torch.zeros((1, 1), dtype=torch.long, device=device)
print(tokenizer.decode(gpt_model.generate(context, max_new_tokens=200)[0].tolist()))

 , ხას კრძალვით და სხვა ასრე ბაგე, რომელი.
 ფრიდონ რამცაშვეწა, ათრთონა, აღარ სიკვდილისა სისხლსა უხები.

 ცუდია ჩემი, ღმერთსა ეელთა ხევა,
 თქვა: “ თუ: ” ჩემი ავდეგო, ყმა თორმეტი უყვარარისა,
 და მიყო, გონება საქებარის რასმე ხრმა ვერ დასმადცაოვსა, ვით ხელმწიფეო, დარბაზს, ვის დამობილისა,
 და მათკე სამსახურად ვეღარა, შეფესა, მაგრა დამლების.

“ ჩემთვის მეკობ უხმო, ყმამან სიტურფე არ ბრძანებდა, თუ ონარ- ნაბ მისლვა რასაცა ორ ამბისა ამერსა;
 ღმერთი მუხლილობასა,
 და შენი რა ვიქმნები, ცრემლი მოეხვია,
“ ვაზირო, სხვად არა ქე- გეუბნიან; ვინ გმირისა, მიჯნურობა თხოს მიწყივ ჩემმან მმართებს ზესების;
 აქანამდის ბნდებიან.
 ვეღარ ვისმცაა, ასმათ სდის, ვარობი,
 მშვილდსა და შეექმნა, სხვა რამე მჭირს, ვპოვე აღარასọ ობლ- თორმეტი ჩამო მივუკვირდა,
 და მე ვით მოამბ მოეხვივნეს, ლომსა თუ᰾


In [ ]:
"""
, ხელი მტკილსა, რა ვუთხარ: არვისგან, ღმერთსა იცი, სხვა დავკარგე გვირგვინი, მამაეცა არ გაიზიდის, აქათების.


 საყვარელსა კაცმან გვის, ტურფაო გულ- აღმაფრეანვენვის მხეცთა ნახვა გარდაჰლდი, არ გამწყვევიდეს, გაიწყინა,
 და ვერ ვერ გავძობასა, უსულოდ გამქნოს, ყოველგნითთქმელი.

 ვიგონებდი, მანდა გა მოვიდიანებისა,
 მას ღმრთისა საუბარი არა გაათაეანთა, მივის დია, ქებალენეგვარა,
 ნაცვლად დაისვა, რათმე ვიკადრებ სავაგლა მევი,
 გკადრო, ამას მის ყმისაობდეს,
 ეტყვის: “ ვა; მკვდართა ხერების, უშამვრისა სოფლისა,
 და მეფეო, მშობელი ვიყავ მას თუ ყორნისნელად რათმე მოჰკვდე, არა ვიცი, ვისიცა მაგისა
 და თუ არსად არ ცნალსა, ათასჯერ ამად არ- ჭვრეტა იშვე,

“ მიბრძანა თუ გველისა უმული,
 და რა მათ აქა ვტირ უკუმ შექმ!

 გავ­,
 კაცსა მამაცნებისაì დაჰხსნია ?”
 მოასხნეს მე
"""